# Task 1: Housing Price Regression

This task is focused on predicting the price of a house listing based on a pre-determined set of features. This notebook is structured as follows:

1. Setup
2. Exploratory Data Analysis (EDA)
3. Model Evaluation
4. Prediction

# 1. Setup

The following section is required to setup this notebook. Import the appropriate modules and modify model hyperparameters here. 

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torch.utils.data import Dataset,DataLoader

import re
from wangsheng_utils import *

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Hyperparameters

RANDOM_CONTROL = 42 # For reproducibility
BEST_MODEL = 'rf' # Choose best regression model based on evaluation
TRAIN_SIZE = 0.8 # Fraction of training set used for model evaluation; the remaining split is validation set
RADIUS_OF_INFLUENCE_KM = 0.5 # Radius of interest for a location centered on their geographical coordinates

# Random Forest: Fill in based on GridSearch results
RF_NUM_ESTIMATORS = 50
RF_MAX_DEPTH = 40
RF_MAX_FEATURES = 2
RF_MIN_SPLIT = 3
RF_MIN_LEAF = 1
RF_BOOTSTRAP = False
RF_CRITERION = "squared_error"

# Gradient Boosting: Fill in based on GridSearch results
GB_NUM_ESTIMATORS = 50
GB_MAX_DEPTH = 50
GB_CRITERION = "squared_error"
GB_LEARNING_RATE = 0.01

# AdaBoost: Fill in based on GridSearch results
AB_NUM_ESTIMATORS = 50
AB_MAX_DEPTH = 30
AB_SPLITTER = "best"
AB_LEARNING_RATE = 0.01

# Extra Trees Regressor: Fill in based on GridSearch results
ET_NUM_ESTIMATORS = 50
ET_MAX_DEPTH = 40
ET_MAX_FEATURES = 2
ET_MIN_SPLIT = 3
ET_MIN_LEAF = 1
ET_BOOTSTRAP = False
ET_CRITERION= "squared_error"

# Bagging Regressor: Fill in based on GridSearch results
BR_NUM_ESTIMATORS = 100
BR_MAX_DEPTH = 50
BR_SPLITTER = "best"
BR_LEARNING_RATE = 0.1

# Neural Net: Fill in based on test iterations
NN_NUM_EPOCHS = 10
NN_BATCH_SIZE = 32
NN_LEARNING_RATE = 0.1

In [ ]:
# Define model training methods
def train(X_feature, y_label, model_name='rf'):
    model = None
    if model_name=='lr':
        model = LinearRegression().fit(X_feature, y_label)
    elif model_name=='rf':
        model = RandomForestRegressor(n_estimators=RF_NUM_ESTIMATORS, criterion=RF_CRITERION, max_depth=RF_MAX_DEPTH,
                      min_samples_split=RF_MIN_SPLIT, min_samples_leaf=RF_MIN_LEAF, max_features=RF_MAX_FEATURES,
                      bootstrap=RF_BOOTSTRAP, random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='gb':
        model = GradientBoostingRegressor(n_estimators=GB_NUM_ESTIMATORS, learning_rate=GB_LEARNING_RATE,
                               max_depth=GB_MAX_DEPTH, criterion=GB_CRITERION, 
                               random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='ab':
        dt = DecisionTreeRegressor(max_depth=AB_MAX_DEPTH, splitter=AB_SPLITTER,
                                   random_state=RANDOM_CONTROL)
        model = AdaBoostRegressor(base_estimator=dt, n_estimators=AB_NUM_ESTIMATORS, learning_rate=AB_LEARNING_RATE,
                              random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='et':
        model = ExtraTreesRegressor(n_estimators=ET_NUM_ESTIMATORS, criterion=ET_CRITERION, max_depth=ET_MAX_DEPTH,
                              min_samples_split=ET_MIN_SPLIT, min_samples_leaf=ET_MIN_LEAF, max_features=ET_MAX_FEATURES,
                              bootstrap=ET_BOOTSTRAP, random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='br':
        dt = DecisionTreeRegressor(max_depth=BG_MAX_DEPTH, splitter=BG_SPLITTER,
                                   random_state=RANDOM_CONTROL)
        model = BaggingRegressor(base_estimator=dt, n_estimators=BR_NUM_ESTIMATORS, learning_rate=BR_LEARNING_RATE,
                             random_state=RANDOM_CONTROL).fit(X_feature, y_label)
    elif model_name=='nn':
        pass
    else:
        pass

    return model
            
# Define model prediction method
def predict(model, X_feature) -> pd.DataFrame: # Fix it. Numpy array is returned
    y_hat = model.predict(X_feature)
    return y_hat

# 2. Exploratory Data Analysis (EDA)

We first explore our given training data. This is done via the following sub-tasks:

- Visualize
- Pre-process
- Post-process

## 2a. Visualize

Here, we take our first look at the data to get an idea of how the attribute values are spread.

In [ ]:
# Read training data
df = pd.read_csv('data/train.csv') 
print(df.shape)
df.head()

In [ ]:
def visualize():
    plt.figure(figsize=(10, 10))
    plt.tick_params(labelsize=10)
    df['price_per_square_ft'] = df['price']/df['size_sqft']
    
    filter_price_sqft_hdb = ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False)))
    df[filter_price_sqft_hdb][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    plot1 = df[filter_price_sqft_hdb][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    
    plt.title("HDB price per square feet($/ft2) vs square feet(ft2)")
    plt.scatter(plot1['price_per_square_ft'],plot1['size_sqft'])
    plt.xlabel('price per square ft ($/ft2)', fontsize=16)
    plt.ylabel('size_sqft (ft2)', fontsize=16)
    plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.tight_layout()
    plt.show()
    
    
    filter_1 = ((df['price_per_square_ft'] < 10000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False))))
    
    plt.figure(figsize=(10, 10))
    plt.tick_params(labelsize=10)
    plot2 = df[filter_1][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    plt.title("HDB price per square feet($/ft2) vs square feet(ft2)")
    plt.scatter(plot2['price_per_square_ft'],plot2['size_sqft'])
    plt.xlabel('price per square ft ($/ft2)', fontsize=16)
    plt.ylabel('size_sqft (ft2)', fontsize=16)
    plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.tight_layout()
    plt.show()
    
    filter_2 = ((df['price_per_square_ft'] < 2000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False))))
    
    plt.figure(figsize=(10, 10))
    plt.tick_params(labelsize=10)
    plot3 = df[filter_2][['property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']]
    plt.title("HDB price per square feet($/ft2) vs square feet(ft2)")
    plt.scatter(plot3['price_per_square_ft'],plot3['size_sqft'])
    plt.xlabel('price per square ft ($/ft2)', fontsize=16)
    plt.ylabel('size_sqft (ft2)', fontsize=16)
    plt.gca().yaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().yaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.gca().xaxis.set_major_formatter(ticker.ScalarFormatter())
    plt.gca().xaxis.set_major_formatter(ticker.StrMethodFormatter("{x}"))
    plt.tight_layout()
    plt.show()
    
    
    filter_3 = ((df['price_per_square_ft'] > 1400) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) & (df.title.str.contains('hdb','Hdb', regex=False))))
    
    table1 = df[filter_3][['title','property_type','num_baths','num_beds','size_sqft','price','price_per_square_ft']].sort_values(by=['price_per_square_ft'])
    pd.options.display.float_format = '{:.0f}'.format
    display(table1)
    
    pass



## 2b. Pre-process

Here, we go through the training data and use the information gained in the section above to pick and choose what attributes we wish to consider in our analysis as well as their representation. 

We call this _pre-processing_ as we are working on the whole training set here. To avoid information leakage, we ensure that we do not look at the label data unless required to check for invalid inputs(more on this below). Additionally, we do not perform any aggregation or imputation here to ensure that the validation set does not influence our training set for the following sections.

The _pre-processing_ is comprised of the following tasks which are performed in order:

- Remove invalid values
- Remove outliers
- Handle missing values
- Data Transform
- Data Augmentation
- Ignore attributes

TODO: Talk about each attribute here. 

For HDB,

Assume that num_beds refers only to the bedrooms excluding the living rooms
* Hdb 2-room = 1 bedroom , 1 bathroom
* Hdb 3-room = 2 bedroom , 2 bathroom
* Hdb 4-room = 3 bedroom , 2 bathroom
* Hdb 5-room = 3 bedroom , 2 bathroom
* Hdb 3-gen = 4 bedroom , 3 bathroom
* Hdb Executive = 3/4 bedroom, 2/3 bathroom
* Hdb Masionette = 3/4 bedroom, 2/3 bathroom
* Hdb Jumbo = 4 bedroom, 4 bathroom

References:
* http://www.data.com.sg/template-m.jsp?p=my/1.html 
* https://www.hdb.gov.sg/residential/buying-a-flat/finding-a-flat/types-of-flats

OneMap:

* https://www.onemap.gov.sg/docs/


In [ ]:
def remove_invalid_values(df) -> pd.DataFrame:
    df = df.copy()
    # Price is the target regression variable. If negative or 0, treat that row as invalid (logical assumption)
    if 'price' in df:
        df = df[df.price > 0]

    # Filter out HDB prices more than 2,000,000 (real-world knowledge; ref. source)
    if 'price' in df:
        filter_price_hdb = ((df.price > 2000000) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
        df_dropped = df[filter_price_hdb]
        df = df.drop(df[filter_price_hdb].index)
        print("Filterout HDB prices more than 2,000,000")
        display(df_dropped)
        
    # Filter out HDB with more bathrooms than bedrooms (real-world knowledge; ref. source)
    filter_bath_beds_hdb = ((df.num_baths > df.num_beds) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df_dropped = df[filter_bath_beds_hdb]
    df = df.drop(df[filter_bath_beds_hdb].index)
    print("Filter out HDB with more bathrooms than bedrooms")
    display(df_dropped)

    # Filter out HDB with more than 4 bathrooms or 6 bedrooms (real-world knowledge; ref. source)
    filter_bath_beds_4_hdb = (((df.num_baths > 4) | (df.num_beds > 6)) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df_dropped = df[filter_bath_beds_4_hdb]
    df = df.drop(df[filter_bath_beds_4_hdb].index)
    print("Filter out HDB with more than 4 bathrooms or 6 bedrooms")
    display(df_dropped)
    
    return df

In [ ]:
def remove_outliers(df) -> pd.DataFrame:
    df = df.copy()
    # Filter out HDB with size > 2500 sqft (outlier detection)
    filter_size_hdb = ((df.size_sqft > 2500) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
    df[filter_size_hdb][['property_type','num_baths','num_beds','size_sqft','price']]
    df_dropped = df[filter_size_hdb]
    df = df.drop(df[filter_size_hdb].index)
    print("Filter out HDB with size > 2500 sqft (outlier detection)")
    display(df_dropped)
    
    # Target label is used below for outlier detection. Appropriately tagged data is dropped entirely.
    if 'price' in df:
    # Filtering those data with less than $200/square feet
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_price_sqft_200 = ((df['price per sq ft'] < 200) & (df['price per sq ft'] > 0))
        df_dropped = df[filter_price_sqft_200]
        df = df.drop(df[filter_price_sqft_200].index)
        print("Filtering those data with less than $200/square feet")
        display(df_dropped)
    # Filtering those data with less than 500 square feet, and more than $5000 per square feet
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_size = ((df['price per sq ft'] > 5000) & (df['price per sq ft'] > 0) & (df['size_sqft'] < 500))
        df_dropped = df[filter_size]
        df = df.drop(df[filter_size].index)
        print("Filtering those data with less than 500 square feet, and more than $5000 per square feet")
        display(df_dropped)
    # Filter out HDB more than $1600 per square feet and since last sold highest price for hdb is $1400
        df['price per sq ft'] = df['price']/df['size_sqft']
        filter_price_hdb_2 = ( (df['price per sq ft']>1600) & ((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=True)) | (df.title.str.contains('hdb','Hdb', regex=False))))
        
        df_dropped = df[filter_price_hdb_2]
        df = df.drop(df[filter_price_hdb_2].index)
        display(df_dropped)
           
        df.drop('price per sq ft', axis=1, inplace=True)
    
    return df

In [ ]:
def handle_missing_values(df, mode='train') -> pd.DataFrame:
    df = df.copy()
    # Treat missing year data as new.
    # Semantically, we define this attribute as the depreciation factor for pricing.
    # A new house or one with missing data denotes the depreciation factor is 0 or unknown.
    # The depreciation factor is assumed to be the difference between construction and current year.
    # TODO: Maybe do not treat future years as current! Inflation factor might be one to look out for.
    df['built_year'] = df['built_year'].fillna(2022)
    
    # TODO: 80 are missing. Should we remove them or should we keep it as 0?
    # Verify assumption if studio qualifies as 1 bed. 
    # 75 of missing are studio, we replace the Nan as 1
    filter_beds_studio = ((df.num_beds.isna()) & ((df.title.str.contains('studio','Studio', flags=re.IGNORECASE, regex=True))))
    df.loc[filter_beds_studio, "num_beds"] = 1
    
    ## HDB
    ## bathrooms
    ## Fill missing data for HDB, bathrooms
         
    ## Fill bathroom = 1 For HDB size 2-room
    df.loc[(df['num_beds'] <= 2) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 1
    ## Fill bathroom = 2 For HDB size 3-room, 4-room, 5-room
    df.loc[(df['num_beds'] < 4) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 2
    ## Fill bathroom = 3 For HDB Executive and Mansionette
    df.loc[(df['num_beds'] == 4) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 3
    ## Fill bathroom = 4 For HDB jumbo
    df.loc[(df['num_beds'] > 4) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 4


    ## If number of beds are missing, then we use size as a gauge

    ## Fill bathroom = 1 For HDB size 500 sq ft and below
    df.loc[(df['size_sqft'] <= 500) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 1
    ## Fill bathroom = 2 For HDB size 1500 sq ft and below
    df.loc[(df['size_sqft'] <= 1500) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 2
    ## Fill bathroom = 3 For HDB size between 1500 sq ft and 2000 sq ft
    df.loc[(df['size_sqft'] > 1500) & (df['size_sqft'] <= 2000) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 3
    ## Fill bathroom = 4 For HDB size between 2000 sq ft and 2500 sq ft
    df.loc[(df['size_sqft'] > 2000) & (df['size_sqft'] <= 2500) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_baths.isna()), "num_baths"] = 4
    
    
    ## HDB
    ## bedrooms
    ## Fill missing data for HDB, bedrooms

    ## Fill bedrooms = 1 For HDB size 500 sq ft and below
    df.loc[(df['num_baths'] == 1) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 2
    ## Fill bedrooms = 2 For HDB size 3-room, 4-room, 5-room
    df.loc[(df['num_baths'] == 2) & (df['size_sqft'] <= 800) &(df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 2
    ## Fill bedrooms = 3 For HDB Executive and Mansionette
    df.loc[(df['num_baths'] == 2) & (df['size_sqft'] <= 1200) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 3
    ## Fill bedrooms = 3 For HDB jumbo
    df.loc[(df['num_baths'] >= 2) & (df['size_sqft'] <= 1500) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 4
    ## Fill bedrooms = 4 For HDB jumbo
    df.loc[(df['num_baths'] > 2) & (df['size_sqft'] <= 2000) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 4
    ## Fill bedrooms = 5 For HDB jumbo
    df.loc[(df['num_baths'] > 3) & (df['size_sqft'] <= 2200) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 5
    ## Fill bedrooms = 6 For HDB jumbo
    df.loc[(df['num_baths'] > 3) & (df['size_sqft'] <= 2500) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 6

    ## If number of baths are missing, then we use size as a gauge

    ## Fill bedrooms = 2 For HDB size 3-room, 4-room, 5-room
    df.loc[(df['size_sqft'] <= 800) &(df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 2
    ## Fill bedrooms = 3 For HDB Executive and Mansionette
    df.loc[(df['size_sqft'] <= 1200) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 3
    ## Fill bedrooms = 3 For HDB jumbo
    df.loc[(df['size_sqft'] <= 1500) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 4
    ## Fill bedrooms = 4 For HDB jumbo
    df.loc[(df['size_sqft'] <= 2000) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 4
    ## Fill bedrooms = 5 For HDB jumbo
    df.loc[(df['size_sqft'] <= 2200) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 5
    ## Fill bedrooms = 6 For HDB jumbo
    df.loc[(df['size_sqft'] <= 2500) & (df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) & (df.num_beds.isna()), "num_beds"] = 6
 
    ## Condos
    ## bathrooms
    ## Fill missing data for condos,apartments bathrooms
         
    ## Fill bathroom = 1 For Condo, apartment size 2-room
    df.loc[(df['num_beds'] <= 2) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 1
    ## Fill bathroom = 2 For Condo, apartment size 2-room size 3-room, 4-room, 5-room
    df.loc[(df['num_beds'] <= 4) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 2
    ## Fill bathroom = 3 For Condo Executive
    df.loc[(df['num_beds'] <= 5) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 3
    ## Fill bathroom = 4 For Huge Condos
    df.loc[(df['num_beds'] > 5) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 4


    ## If number of beds are missing, then we use size as a gauge

    ## Fill bathroom = 1 For Condos less than 800 sq ft
    df.loc[(df['size_sqft'] <= 800) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 1
    ## Fill bathroom = 2 For Condos between 800 sq ft and 1300 sq ft
    df.loc[(df['size_sqft'] <= 1300) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 2
    ## Fill bathroom = 3 For Condos between 1300 sq ft and 1600 sq ft
    df.loc[(df['size_sqft'] <= 1600) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 3
    ## Fill bathroom = 4 For Huge Condos
    df.loc[(df['size_sqft'] > 1600) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_baths.isna()), "num_baths"] = 4
    
    ## Condos
    ## bedrooms
    
    ## Fill bedrooms = 2  For Condos between 800 sq ft and 1300 sq ft
    df.loc[(df['size_sqft'] <= 800) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_beds.isna()), "num_beds"] = 2
    ## Fill bedrooms = 3  For Condos between 850 sq ft and 1250 sq ft
    df.loc[(df['size_sqft'] <= 1250) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_beds.isna()), "num_beds"] = 3
    ## Fill bedrooms = 4  For Condos between 1250 sq ft and 1500 sq ft
    df.loc[(df['size_sqft'] <= 1500) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_beds.isna()), "num_beds"] = 4
    ## Fill bedrooms = 5  For Condos more than 1500 sq ft
    df.loc[(df['size_sqft'] > 1500) & ((df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False))) & (df.num_beds.isna()), "num_beds"] = 5
    
    ## All other houses
    ## bedrooms
    
    ## Fill bedrooms = 2  For houses between 800 sq ft and 1300 sq ft
    df.loc[(df['size_sqft'] <= 800) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_beds.isna()), "num_beds"] = 2
    ## Fill bedrooms = 3  For houses between 850 sq ft and 1250 sq ft
    df.loc[(df['size_sqft'] <= 1250) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_beds.isna()), "num_beds"] = 3
    ## Fill bedrooms = 4  For houses between 1250 sq ft and 1500 sq ft
    df.loc[(df['size_sqft'] <= 1500) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_beds.isna()), "num_beds"] = 4
    ## Fill bedrooms = 5  For houses more than 1500 sq ft
    df.loc[(df['size_sqft'] > 1500) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_beds.isna()), "num_beds"] = 5
    
    ## All other houses
    ## bathrooms
    
    ## Fill bathrooms = 1  For houses with 1 bed
    df.loc[(df['num_beds'] == 1) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_baths.isna()), "num_baths"] = 1
    ## Fill bathrooms = 2  For houses with 2 beds
    df.loc[(df['num_beds'] == 2) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_baths.isna()), "num_baths"] = 2
    ## Fill bedrooms = 2  For houses with 3 beds
    df.loc[(df['num_beds'] == 3 ) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_baths.isna()), "num_baths"] = 2
    ## Fill bedrooms = 3  For houses with 4 beds
    df.loc[(df['num_beds'] == 4) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_baths.isna()), "num_baths"] = 3
    ## Fill bedrooms = 4  For houses with 5 or more beds
    df.loc[(df['num_beds'] > 4) & (~((df.property_type.str.contains('hdb','Hdb', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('condo','Condo', flags=re.IGNORECASE, regex=False)) | (df.property_type.str.contains('apart','Apart', flags=re.IGNORECASE, regex=False)))) & (df.num_baths.isna()), "num_baths"] = 4
    
    if mode=='train':
        # 5 of missing, we do not have much info. Dropping these 5 data
        filter_drop_beds = df.num_beds.isna()
        df_dropped = df[filter_drop_beds]
        df = df.drop(df[filter_drop_beds].index)
        print("Dropped missing beds")
        display(df_dropped)

        # TODO: 400 are missing. Cannot remove so many data. Use 0 to denote absence of attribute.
        filter_drop_baths = df.num_baths.isna()
        df = df.drop(df[filter_drop_baths].index)
    elif mode=='test':
        

        # TODO: Filling with 0 for test data. Some imputation might be better.
        df['num_beds'] = df['num_beds'].fillna(0)
        df['num_baths'] = df['num_baths'].fillna(0)
                
    # Handle missing coordinates data
    for idx in df.index:
        if pd.isnull(df.at[idx, 'lat']) or pd.isnull(df.at[idx, 'lng']) or pd.isnull(df.at[idx, 'subzone']):
            lat, lng = find_lat_lng(df.at[idx, 'address'])
            if lat is not None:
                df.at[idx, 'lat'] = float(lat)
                #print("Fixed address {} with latitude: {}".format(df.at[idx, 'address'], df.at[idx, 'lat']))
            if lng is not None:
                df.at[idx, 'lng'] = float(lng)
                #print("Fixed address {} with longitude: {}".format(df.at[idx, 'address'], df.at[idx, 'lng']))
    
    return df

In [ ]:
def transform_data(df) -> pd.DataFrame:
    df = df.copy()
    # TODO: Test against not doing this.
    df.loc[df["built_year"] > 2022, "built_year"] = 2022
    
    # Convert built_year into the aforementioned depreciation factor
    df["depreciation"] = (2022-df["built_year"])
    '''
    # TODO: Add details on why we are doing so
    df.loc[df.property_type.str.contains('hdb', flags=re.IGNORECASE, regex=True), 'property_type'] = 'hdb'
    df.loc[df.property_type.str.contains('condo', flags=re.IGNORECASE, regex=True), 'property_type'] = 'condo'
    df['property_type'] = df['property_type'].str.lower()
    df.loc[df.property_type.str.contains('cluster house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('townhouse', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('land only', flags=re.IGNORECASE, regex=True), 'property_type'] = 'landed'
    df.loc[df.property_type.str.contains('apartment',  flags=re.IGNORECASE, regex=True), 'property_type'] = 'condo'
    df.loc[df.property_type.str.contains('bungalow', flags=re.IGNORECASE, regex=True), 'property_type'] = 'bungalow'
    df.loc[df.property_type.str.contains('semi-detached house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'corner'
    df.loc[df.property_type.str.contains('corner terrace',flags=re.IGNORECASE, regex=True), 'property_type'] = 'corner'
    df.loc[df.property_type.str.contains('shophouse', flags=re.IGNORECASE, regex=True), 'property_type'] = 'protected'
    df.loc[df.property_type.str.contains('conservation house', flags=re.IGNORECASE, regex=True), 'property_type'] = 'protected'
    
    # Get one hot encoding of columns property_type
    one_hot = pd.get_dummies(df['property_type'])
    # Join the encoded df
    df = df.join(one_hot)
    '''
    '''
    df['tenure'] = df['tenure'].fillna(value=df.property_type)
    df.loc[df.tenure.str.contains('hdb', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('condo', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('terraced house', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('corner', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('landed', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('protected', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('bungalow', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'

    df.loc[df.tenure.str.contains('110-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('103-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('102-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'
    df.loc[df.tenure.str.contains('100-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = '99-year leasehold'

    df.loc[df.tenure.str.contains('999-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('946-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('956-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('947-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'
    df.loc[df.tenure.str.contains('929-year leasehold', flags=re.IGNORECASE, regex=True), 'tenure'] = 'freehold'

    df['encoded_tenure'] = 0
    df.loc[df.tenure.str.contains('freehold', flags=re.IGNORECASE, regex=True), 'encoded_tenure'] = 1
    '''
    
    df['encoded_furnishing'] = 0
    df.loc[df.furnishing.str.contains('partial', flags=re.IGNORECASE, regex=True), 'encoded_furnishing'] = 0.5
    df.loc[df.furnishing.str.contains('unfurnished', flags=re.IGNORECASE, regex=True), 'encoded_furnishing'] = -1
    df.loc[df.furnishing.str.contains('fully', flags=re.IGNORECASE, regex=True), 'encoded_furnishing'] = 1

    return df

In [ ]:
def augment_data(df, distance=1) -> pd.DataFrame:
    df = df.copy()
    # get the files containing the infrastructures data
    cc = pd.read_csv("data/auxiliary-data/sg-commerical-centres.csv")
    mrt = pd.read_csv("data/auxiliary-data/sg-mrt-stations.csv")
    ps = pd.read_csv("data/auxiliary-data/sg-primary-schools.csv")
    ss = pd.read_csv("data/auxiliary-data/sg-secondary-schools.csv")
    sm = pd.read_csv("data/auxiliary-data/sg-shopping-malls.csv")
    sz = pd.read_csv("data/auxiliary-data/sg-subzones.csv")

    # calculate distance to nearest commercial center
    df['dist_2_nearest_cc'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], cc), 
        axis=1)
    df['dist_2_nearest_cc'] = df['dist_2_nearest_cc'].round(decimals=3)

    # calculate the number of commercial centers within x km
    df['nearest_cc_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], cc, distance), 
        axis=1)
    
    # calculate distance to nearest mrt station
    df['dist_2_nearest_mrt'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], mrt), 
        axis=1)
    df['dist_2_nearest_mrt'] = df['dist_2_nearest_mrt'].round(decimals=3)

    # calculate the number of mrt stations within x km
    df['nearest_mrt_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], mrt, distance), 
        axis=1)
    
    # calculate distance to nearest primary school
    df['dist_2_nearest_ps'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], ps), 
        axis=1)
    df['dist_2_nearest_ps'] = df['dist_2_nearest_ps'].round(decimals=3)

    # calculate the number of primary schools within x km
    df['nearest_ps_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], ps, distance), 
        axis=1)
    # calculate distance to nearest secondary school
    df['dist_2_nearest_ss'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], ss), 
        axis=1)
    df['dist_2_nearest_ss'] = df['dist_2_nearest_ss'].round(decimals=3)

    # calculate the number of secondary schools within x km
    df['nearest_ss_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], ss, distance), 
        axis=1)
    
    # calculate distance to nearest shopping mall
    df['dist_2_nearest_sm'] = df.apply(
        lambda row: find_nearest_distance(row['lat'], row['lng'], sm), 
        axis=1)
    df['dist_2_nearest_sm'] = df['dist_2_nearest_sm'].round(decimals=3)

    # calculate the number of secondary schools within x km
    df['nearest_sm_count'] = df.apply(
        lambda row: count_nearest(row['lat'], row['lng'], sm, distance), 
        axis=1)

    return df

In [ ]:
def ignore_attributes(df) -> pd.DataFrame:
    df = df.copy()
    # Drop listing id; nominal identifier with no meaning
    df.drop('listing_id', axis=1, inplace=True)

    # Drop elevation; all the values are 0, spurious attribute
    df.drop('elevation', axis=1, inplace=True)

    # Drop url; nominal identifier with no meaning; useful for manual lookups or scraping
    df.drop('property_details_url', axis=1, inplace=True)

    # Drop floor level as 83% missing and sparse with the rest of the values. 
    # Not enough data available to get the model trained.
    df.drop('floor_level', axis=1, inplace=True)

    # Drop column property_type and tenure as it is now encoded
    df.drop('property_type',axis = 1, inplace=True)
    df.drop('tenure',axis=1, inplace=True)

    # Drop latitude and longitude; using it in conjunction with auxiliary data
    # Actual values are not semantically useful
    df.drop('lat', axis=1, inplace=True)
    df.drop('lng', axis=1, inplace=True)
    # Drop planning area to avoid dimensional bloat
    # Categories replaced with numerical identifiers from auxiliary data
    df.drop('planning_area', axis=1, inplace=True)
    # Address used in conjunction with augmentation
    df.drop('address', axis=1, inplace=True)

    # Drop this attribute as a significant amount of data(27.9%) is missing here.
    # We are unable to make much sense of this attribute.
    df.drop('total_num_units', axis=1, inplace=True) 

    # Transformed year into a depreciation factor already. Drop original attribute.
    df.drop('built_year', axis=1, inplace=True)

    # Title data is non-standard and unstructured with information already available in individual attributes.
    df.drop('title', axis=1, inplace=True)
    
    # Property name is assumed to have no inherent effect on the value of a listing.
    df.drop('property_name', axis=1, inplace=True)

    # BELOW CODE IN THIS SECTION IS ONLY MEANT TO GET THE SKELETON WORKING; RE-EVALUATE EACH ATTRIBUTE ONE BY ONE
    df.drop('furnishing', axis=1, inplace=True)
    df.drop('available_unit_types', axis=1, inplace=True)

    return df

In [ ]:
def pre_process(df, mode='train') -> pd.DataFrame:
    if mode=='train': 
        df = remove_invalid_values(df)
        df = remove_outliers(df)
    df = handle_missing_values(df, mode)
    df = transform_data(df)
    df = augment_data(df, distance=RADIUS_OF_INFLUENCE_KM)
    df = ignore_attributes(df)
    
    return df

In [ ]:
# You may skip this block if you are not interested in Section 3.

df_preprocessed = pre_process(df, mode='train')
print(df_preprocessed.shape)
df_preprocessed.head()

## 2c. Post-process

Here, we separately process the training and validation sets. Since we perform aggregation-based methods here, this is done after split to avoid information leakage and ensures salient evaluation results.

If you wish to explore each model and go through our data handling, proceed to Section 3 - Model Evaluation. Alternately, if you are only interested in the final output, you may jump to Section 4 - Prediction to gather results.

In [ ]:
def target_encoding_location(df, mode='train', aggregate=None):
    # Target encode subzone based on price.
    cols = ('subzone')
    df.loc[:,cols] = df.loc[:,cols].fillna('null')
    if mode == 'train':
        aggregate = df['price'].groupby(df.loc[:,cols]).median()
    for idx in df.index:
        if df.at[idx, cols] in aggregate:
            df.at[idx, cols] = float(aggregate[df.at[idx, cols]])
        else:
            df.at[idx, cols] = float(aggregate['null'])
    return df, aggregate

def standardize(df, mode='train', scaler=None):
    # Normalize values here.
    cols = ('size_sqft', 'dist_2_nearest_cc', 'dist_2_nearest_mrt', 'dist_2_nearest_ps', 'dist_2_nearest_ss', 'dist_2_nearest_sm', 'subzone')
    if scaler is None:
        scaler = StandardScaler()
    if mode=='train':
        df.loc[:,cols] = scaler.fit_transform(df.loc[:,cols])
    elif mode=='test':
        df.loc[:,cols] = scaler.transform(df.loc[:,cols])
    return df, scaler
    

def post_process(df, mode='train', aggregate=None, scaler=None):
    df, aggregate = target_encoding_location(df, mode, aggregate)
    df, scaler = standardize(df, mode, scaler)
    return df, aggregate, scaler


In [ ]:
# You may skip this block if you are not interested in Section 3.

# First we create our train and validation split to ensure there is no information leakage from validation set to train set
train_set, val_set = train_test_split(df_preprocessed, train_size=TRAIN_SIZE, random_state=RANDOM_CONTROL, shuffle=True)

# Then, we run post-processing on train set and use it to post-process validation set.
train_set, aggregate, scaler = post_process(train_set, mode='train')
val_set, _, _ = post_process(val_set, mode='test', aggregate=aggregate, scaler=scaler)

# Check for why we are removing Mandai Estate!!

# 3. Model Evaluation

Here, we evaluate models for their predictions. RMSE is the metric used for comparison. The following models are evaluated:

* Linear Regression
* Random Forest
* Gradient Boosting
* AdaBoost
* Extra Trees Regressor
* Bagging Regressor
* Neural Net

This section also involves hyperparameter tuning wherein we search for best settings per model. Those settings are later used to set hyperparameters as in Section 1 and dictate the final predictions in Section 4. 

In [ ]:
def validate(y, y_hat) -> None:
    rmse = mean_squared_error(y, y_hat, squared=False)
    print('Validation RMSE: {:.3}'.format(rmse))
    return

# Need to recreate whole set for Section 3. This is a 'hack' to get GridSearch to work with cross-validation.
# To ensure that we get consistent results across multiple iterations, we need to fix the train-test split.
# Later we run GridSearch on entire set by passing in our predefined split as parameter. 
df_postprocessed = pd.concat([train_set, val_set], ignore_index=True)
X_housing = df_postprocessed.loc[:, df_postprocessed.columns != 'price']
y_housing = df_postprocessed['price']
X_train = train_set.loc[:, train_set.columns != 'price']
y_train = train_set['price']
X_val = val_set.loc[:, val_set.columns != 'price']
y_val = val_set['price']

# Use our predefined split for GridSearch below.
val_split_indices = [-1 if x in X_train.index else 0 for x in X_housing.index]
ps = PredefinedSplit(test_fold=val_split_indices)

## 3a. Linear Regression

In [ ]:
model_lr = train(X_train, y_train, model_name='lr')
y_hat_lr = predict(model_lr, X_val)
validate(y_val, y_hat_lr)

## 3b. Random Forest

In [ ]:
def bestfit_rf(X_feature, y_label, train_test_split):
    estimator = RandomForestRegressor()
    params = {'n_estimators': [25, 50, 100, 200],
              'max_depth': [5, 10, 20, 30, 40, 50],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4],
              'criterion': ["squared_error"],
              'max_features': [1, 2, 3],
              'bootstrap': [True, False],
              'random_state': [RANDOM_CONTROL]}
    model_rf = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_rf.fit(X_feature, y_label)
    print('Random Forest Best Parameters: {}'.format(model_rf.best_params_))
    return model_rf

model_rf = bestfit_rf(X_housing, y_housing, ps)
y_hat_rf = predict(model_rf, X_val)
validate(y_val, y_hat_rf)

## 3c. Gradient Boosting

In [ ]:
def bestfit_gb(X_feature, y_label, train_test_split):
    estimator = GradientBoostingRegressor()
    params = {'n_estimators': [25, 50, 100, 200],
              'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
              'max_depth': [5, 10, 20, 30, 40, 50],
              'min_samples_split': [2, 3, 4, 5],
              'min_samples_leaf': [1, 2, 3, 4],
              'criterion': ["squared_error", "friedman_mse"],
              'max_features': [1, 2, 3],
              'random_state': [RANDOM_CONTROL]}
    model_gb = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_gb.fit(X_feature, y_label)
    print('Gradient Boost Best Parameters: {}'.format(model_gb.best_params_))
    return model_gb
    

model_gb = bestfit_gb(X_housing, y_housing, ps)
y_hat_gb = predict(model_gb, X_val)
validate(y_val, y_hat_gb)

## 3d. AdaBoost

In [ ]:
def bestfit_ab(X_feature, y_label, train_test_split):
    base_estimator = DecisionTreeRegressor()
    estimator = AdaBoostRegressor(base_estimator=base_estimator)
    params = {'base_estimator__max_depth': [5, 10, 20, 30, 40, 50],
              'base_estimator__splitter': ['best', 'random'],
              'n_estimators': [25, 50, 100, 200],
              'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.5, 1],
              'random_state': [RANDOM_CONTROL]}
    model_ab = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_ab.fit(X_housing, y_housing)
    print('AdaBoost Best Parameters: {}'.format(model_ab.best_params_))
    return model_ab


model_ab = bestfit_ab(X_housing, y_housing, ps)
y_hat_ab = predict(model_ab, X_val)
validate(y_val, y_hat_ab)

## 3e. Extra Trees Regressor

In [ ]:
def bestfit_et(X_feature, y_label, train_test_split):
    estimator = ExtraTreesRegressor()
    params = {'n_estimators': [25, 50, 100],
              'max_depth': [5, 10, 25, 50],
              'min_samples_split': [2],
              'min_samples_leaf': [1],
              'criterion': ["squared_error", "friedman_mse"],
              'max_features': [1],
              'bootstrap': [True, False],
              'random_state': [RANDOM_CONTROL]}
    model_et = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=train_test_split)
    model_et.fit(X_housing, y_housing)
    print('Extra Trees Best Parameters: {}'.format(model_et.best_params_))
    return model_et


model_et = bestfit_et(X_housing, y_housing, ps)
y_hat_et = predict(model_et, X_val)
validate(y_val, y_hat_et)

## 3f. Bagging Regressor

In [ ]:
def bestfit_br(X_feature, y_label, train_test_split):
    base_estimator = DecisionTreeRegressor()
    estimator = BaggingRegressor(base_estimator=base_estimator)
    params = {'n_estimators': [25, 50, 100],
              'max_features': [1],
              'random_state': [RANDOM_CONTROL]}
    model_br = GridSearchCV(estimator=estimator,
                         param_grid=params,
                         cv=ps)
    model_br.fit(X_housing, y_housing)
    print('Bagging Best Parameters: {}'.format(model_br.best_params_))
    return model_br


model_br = bestfit_br(X_housing, y_housing, ps)
y_hat_br = predict(model_br, X_val)
validate(y_val, y_hat_br)

## 3g. Neural Net

In [ ]:
X_housing = X_housing.to_numpy()
y_housing = y_housing.to_numpy()
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_val = X_val.to_numpy()
y_val = y_val.to_numpy()

#print(X_train.shape)
#print(y_train.shape)
train_dataloader = DataLoader([ [X_train[i], y_train[i]] for i in range(len(X_train)) ], batch_size=NN_BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# Define MLP architecture
class Model(nn.Module):
    
    def __init__(self, device='cpu'):
        super(Model, self).__init__()
        self.device = device
        
        # Modify accordingly
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 16)
        self.fc3 = nn.Linear(16, 64)
        self.fc4 = nn.Linear(64, 4)
        self.fc5 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        
        self.dense = nn.Sequential(self.fc1, self.relu, self.fc2, self.relu, self.fc3, self.relu, 
                                   self.fc4, self.relu, self.fc5)
        
    def forward(self, x):
        pred = self.dense(x)
        return pred
    
# Train
device = 'cpu'
model = Model()
optimizer = optim.Adam(model.parameters(), NN_LEARNING_RATE)
criterion = nn.MSELoss()
model.to(device)
for epoch in range(NN_NUM_EPOCHS):
    running_loss = 0
    for idx, (x_features, y_labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        x_features = x_features.to(device, dtype=torch.float)
        y_labels = y_labels.to(device, dtype=torch.float)
        prediction = model(x_features)
        loss = torch.sqrt(criterion(prediction, y_labels)) # Standardize RMSE loss
        loss.backward()
        optimizer.step()
        running_loss += loss
        if (idx+1) %100 == 0: 
            running_loss = format(running_loss/100, '.4f')
            print(f"Epoch [{epoch+1} Batches processed | {idx}] Loss: {running_loss}")
            running_loss = 0
print("Finished Training.")


In [ ]:
# Validate
X_features = torch.from_numpy(X_val)
y_labels = torch.from_numpy(y_val)
X_features = X_features.to(device, dtype=torch.float)
y_labels = y_labels.to(device, dtype=torch.float)
prediction = model(X_features)
rmse_nn = torch.sqrt(criterion(prediction, y_labels))

print('Neural Net Validation rmse: {:.3}'.format(rmse_nn))

# 4. Prediction

Now that we are done evaluating each model and comparing their performance, we select the model that performed best on the validation set to generate our overall predictions. We re-train this model from scratch on the whole set of training data here as it is assumed ready to ship so we wish to use as much data for training as we can. We later test this model against the provided test data and generate a CSV file of predicted values.

In [ ]:
# Train over entire training set
df_train = pd.read_csv('data/train.csv') 
df_train = pre_process(df_train, mode='train')
df_train, aggregate, scaler = post_process(df_train, mode='train')
X_train = df_train.loc[:, df_train.columns != 'price']
y_train = df_train['price']
model = train(X_train, y_train, model_name=BEST_MODEL) # Choose your model from {'lr', 'rf', 'gb', ab', 'et', 'br', 'nn'}

# Predict labels for test set
df_test = pd.read_csv('data/test.csv') 
df_test = pre_process(df_test, mode='test')
df_test, _, _ = post_process(df_test, mode='test', aggregate=aggregate, scaler=scaler)
predictions = predict(model, df_test)
predictions = pd.DataFrame(predictions)
predictions.to_csv('predictions.csv', header=['Predicted'], index=True, index_label='Id')